# Cloths recomendation system design 🤖 --> 👚
LangGraph and ollama 3.2 agentic AI system is used for cloth recomendation system.

In [4]:
!pip install langchain_ollama

In [1]:
import subprocess
import time
import os

# 1. Install Ollama (this step is blocking by nature)
print("Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

Installing Ollama...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tgz
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


#Setting Up ollama chat langchain

In [2]:
import subprocess
import time
import asyncio
import nest_asyncio

# Necessary for Jupyter/Notebook environments
nest_asyncio.apply()
def setup_ollama():
    # 1. Start the Ollama server in the background
    print("Starting Ollama server...")
    ollama_log = open("ollama_server.log", "w")

    # Use start_new_session to ensure the server keeps running independently
    subprocess.Popen(
        ["ollama", "serve"],
        stdout=ollama_log,
        stderr=ollama_log,
        start_new_session=True
    )

    # 2. Give the server a moment to initialize
    time.sleep(5)

    # 3. Pull the model (Blocking call to ensure it's ready before use)
    print("Downloading Llama llama3.2... Please wait, this may take a few minutes.")
    try:
        subprocess.run(["ollama", "pull", "llama3.2"], check=True)
        print("✅ Model downloaded and server is ready!")
    except subprocess.CalledProcessError as e:
        print(f"❌ Error pulling model: {e}")

if __name__ == "__main__":
    setup_ollama()
    # Your weather or LLM logic goes here

Starting Ollama server...
✅ Model downloaded and server is ready!


##Ollama example test

In [5]:
import asyncio
from langchain_ollama import ChatOllama

# Small model for fast response
llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    num_thread=4,
    max_tokens=50
)

async def main():
    answer = ""
    print("Streaming response:\n")
    async for chunk in llm.astream("What is the capital of Iran?"):
        # Each chunk has .content
        print(chunk.content, end="", flush=True)
        answer += chunk.content

    print("\n\nFinal answer:", answer)

await main()


Streaming response:

The capital of Iran is Tehran.

Final answer: The capital of Iran is Tehran.


#Setting up the weather APIs
### First setup Geolocation finder

In [6]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapi")
location = geolocator.geocode("Paris")
print(location.latitude, location.longitude)


48.8588897 2.320041


##Setting up weather fetcher API function

In [7]:
import asyncio
import httpx
import time
from typing import Dict
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable

# -----------------------------
# Weather cache
# -----------------------------
WEATHER_CACHE = {}
TTL_SECONDS = 600  # 10 minutes cache

# -----------------------------
# City coordinates cache
# -----------------------------
CITY_CACHE = {}

# -----------------------------
# Geopy geocoder
# -----------------------------
geolocator = Nominatim(user_agent="geoapi")

async def get_city_coords(city: str, max_retries=5) -> tuple:
    """Get city coordinates with retry, caching, and throttling"""
    if city in CITY_CACHE:
        return CITY_CACHE[city]

    for attempt in range(max_retries):
        try:
            loop = asyncio.get_event_loop()
            location = await loop.run_in_executor(None, lambda: geolocator.geocode(city, timeout=10))
            if location:
                coords = (location.latitude, location.longitude)
                CITY_CACHE[city] = coords
                # Respect Nominatim usage policy: 1 request per second
                await asyncio.sleep(1)
                return coords
        except (GeocoderTimedOut, GeocoderUnavailable):
            # exponential backoff
            await asyncio.sleep(2 ** attempt)

    raise ValueError(f"City '{city}' could not be geocoded after {max_retries} retries!")

# -----------------------------
# Async fetch weather
# -----------------------------
async def fetch_weather(lat: float, lon: float) -> Dict:
    key = (lat, lon)
    now = time.time()

    if key in WEATHER_CACHE:
        data, ts = WEATHER_CACHE[key]
        if now - ts < TTL_SECONDS:
            return data

    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    async with httpx.AsyncClient(timeout=10) as client:
        resp = await client.get(url)
        resp.raise_for_status()
        data = resp.json()

    WEATHER_CACHE[key] = (data, now)
    return data

# -----------------------------
# Get weather + clothing advice
# -----------------------------
async def get_weather_and_clothing(city: str) -> Dict:
    start_time = time.time()

    lat, lon = await get_city_coords(city)
    data = await fetch_weather(lat, lon)

    current = data.get("current_weather", {})
    temp = current.get("temperature", 20)
    code = current.get("weathercode", 0)

    code_to_desc = {
        0: "Clear", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
        45: "Fog", 48: "Rime fog", 51: "Drizzle", 53: "Drizzle", 55: "Drizzle",
        61: "Rain", 63: "Rain", 65: "Rain", 71: "Snow", 73: "Snow", 75: "Snow",
        80: "Rain showers", 81: "Rain showers", 82: "Rain showers", 95: "Thunderstorm",
        96: "Thunderstorm with hail", 99: "Thunderstorm with hail"
    }
    description = code_to_desc.get(code, "Clear")

    if temp < 5:
        advice = "Freezing! Heavy coat, thermal layers, and gloves."
    elif 5 <= temp < 15:
        advice = "Chilly. Light jacket or sweater."
    elif 15 <= temp < 22:
        advice = "Mild. Light sweater or denim jacket."
    else:
        advice = "Warm. T-shirt and shorts."

    if "rain" in description.lower():
        advice += " Bring an umbrella!"

    elapsed = time.time() - start_time
    print(f"[{city}] Fetched in {elapsed:.3f} sec")

    return {
        "city": city,
        "temperature": f"{temp}°C",
        "condition": description,
        "advice": advice
    }

# -----------------------------
# Run for multiple cities sequentially to respect rate limits
# -----------------------------
async def main():
    cities = ["New York", "London", "Paris", "Toronto", "Berlin", "Tokyo", "Sydney"]
    results = []
    for city in cities:
        try:
            results.append(await get_weather_and_clothing(city))
        except ValueError as e:
            print(e)

    print("\n--- Results ---")
    for r in results:
        print(r)

# -----------------------------
# Run
# -----------------------------
await main()


[New York] Fetched in 3.845 sec
[London] Fetched in 3.427 sec
[Paris] Fetched in 2.605 sec
[Toronto] Fetched in 2.033 sec
[Berlin] Fetched in 2.503 sec
[Tokyo] Fetched in 2.105 sec
[Sydney] Fetched in 2.732 sec

--- Results ---
{'city': 'New York', 'temperature': '7.5°C', 'condition': 'Overcast', 'advice': 'Chilly. Light jacket or sweater.'}
{'city': 'London', 'temperature': '3.1°C', 'condition': 'Overcast', 'advice': 'Freezing! Heavy coat, thermal layers, and gloves.'}
{'city': 'Paris', 'temperature': '4.4°C', 'condition': 'Overcast', 'advice': 'Freezing! Heavy coat, thermal layers, and gloves.'}
{'city': 'Toronto', 'temperature': '3.2°C', 'condition': 'Overcast', 'advice': 'Freezing! Heavy coat, thermal layers, and gloves.'}
{'city': 'Berlin', 'temperature': '-0.9°C', 'condition': 'Overcast', 'advice': 'Freezing! Heavy coat, thermal layers, and gloves.'}
{'city': 'Tokyo', 'temperature': '12.4°C', 'condition': 'Mainly clear', 'advice': 'Chilly. Light jacket or sweater.'}
{'city': 'Syd

In [17]:
import asyncio
import httpx
import time
from typing import Dict
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable

from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_ollama import ChatOllama
import nest_asyncio

# -----------------------------
# Notebook/async fix
# -----------------------------
nest_asyncio.apply()

# -----------------------------
# Caches
# -----------------------------
WEATHER_CACHE = {}
CITY_CACHE = {}
TTL_SECONDS = 600  # 10 minutes cache

# -----------------------------
# Geopy geocoder
# -----------------------------
geolocator = Nominatim(user_agent="fashion_advisor")

async def get_city_coords(city: str, max_retries=5) -> tuple:
    """Get city coordinates with retry, caching, and throttling"""
    if city in CITY_CACHE:
        return CITY_CACHE[city]

    for attempt in range(max_retries):
        try:
            loop = asyncio.get_event_loop()
            location = await loop.run_in_executor(None, lambda: geolocator.geocode(city, timeout=10))
            if location:
                coords = (location.latitude, location.longitude)
                CITY_CACHE[city] = coords
                # Respect Nominatim usage policy: 1 request per second
                await asyncio.sleep(1)
                return coords
        except (GeocoderTimedOut, GeocoderUnavailable):
            await asyncio.sleep(2 ** attempt)

    raise ValueError(f"City '{city}' could not be geocoded after {max_retries} retries!")

# -----------------------------
# Async fetch weather
# -----------------------------
async def fetch_weather(lat: float, lon: float) -> Dict:
    key = (lat, lon)
    now = time.time()

    if key in WEATHER_CACHE:
        data, ts = WEATHER_CACHE[key]
        if now - ts < TTL_SECONDS:
            return data

    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    async with httpx.AsyncClient(timeout=10) as client:
        resp = await client.get(url)
        resp.raise_for_status()
        data = resp.json()

    WEATHER_CACHE[key] = (data, now)
    return data

# -----------------------------
# Weather + clothing advice tool
# -----------------------------
@tool
async def get_weather_and_clothing(city: str) -> Dict:
    """
    Fetch the current weather for a city and provide clothing advice
    based on temperature and weather conditions.
    """
    start_time = time.time()

    lat, lon = await get_city_coords(city)
    data = await fetch_weather(lat, lon)

    current = data.get("current_weather", {})
    temp = current.get("temperature", 20)
    code = current.get("weathercode", 0)

    code_to_desc = {
        0: "Clear", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
        45: "Fog", 48: "Rime fog", 51: "Drizzle", 53: "Drizzle", 55: "Drizzle",
        61: "Rain", 63: "Rain", 65: "Rain", 71: "Snow", 73: "Snow", 75: "Snow",
        80: "Rain showers", 81: "Rain showers", 82: "Rain showers", 95: "Thunderstorm",
        96: "Thunderstorm with hail", 99: "Thunderstorm with hail"
    }
    description = code_to_desc.get(code, "Clear")

    if temp < 5:
        advice = "Freezing! Heavy coat, thermal layers, and gloves."
    elif 5 <= temp < 15:
        advice = "Chilly. Light jacket or sweater."
    elif 15 <= temp < 22:
        advice = "Mild. Light sweater or denim jacket."
    else:
        advice = "Warm. T-shirt and shorts."

    if "rain" in description.lower():
        advice += " Bring an umbrella!"

    elapsed = time.time() - start_time
    print(f"[{city}] Fetched in {elapsed:.3f} sec")

    return {
        "city": city,
        "temperature": f"{temp}°C",
        "condition": description,
        "advice": advice
    }


# -----------------------------
# All tools
# -----------------------------
all_tools = [get_weather_and_clothing]

# -----------------------------
# LLM init
# -----------------------------
llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    num_thread=4
).bind_tools(all_tools)

# -----------------------------
# Graph setup
# -----------------------------
async def chatbot(state: MessagesState):
    system_prompt = SystemMessage(content=(
        "You are a stylish Fashion Advisor. Use the 'get_weather_and_clothing' tool. "
        "Take the base advice provided by the tool and turn it into a friendly, "
        "high-end fashion recommendation for the user.  In plain English and brief pointing to the important tips"
    ))
    messages = [system_prompt] + state["messages"]
    response = await llm.ainvoke(messages)
    return {"messages": [response]}

builder = StateGraph(MessagesState)
builder.add_node("agent", chatbot)
builder.add_node("tools", ToolNode(all_tools))

builder.add_edge(START, "agent")
builder.add_conditional_edges("agent", tools_condition)
builder.add_edge("tools", "agent")

graph = builder.compile()

# -----------------------------
# Execution example
# -----------------------------
async def main():
    start = time.time()
    cities = ["Montreal", "London", "New York"]
    inputs = {"messages": [("user", f"I'm in {cities[0]}. What should I wear today?")]}

    async for event in graph.astream(inputs, stream_mode="values"):
        if "messages" in event:
            last_message = event["messages"][-1]
            if last_message.type == "ai" and not last_message.tool_calls:
                print("\nADVISOR RESPONSE:")
                print(last_message.content)
    end = time.time()
    print('\n'+ '*'* 300 +'\n')
    print(f"Total response time: {end - start}")
'''
    print("\n--- Multi-city Weather Fetch ---")
    # Use .ainvoke with a dictionary for tools
    results = await asyncio.gather(*(get_weather_and_clothing.ainvoke({"city": city}) for city in cities))
    for r in results:
        print(r)
'''
# -----------------------------
# Run
# -----------------------------
await main()


[Montreal] Fetched in 2.347 sec

ADVISOR RESPONSE:
Bonjour! Given the current weather conditions in Montreal, I would recommend dressing warmly and stylishly for your day.

First, start with a foundation of thermal layers - think long-sleeved tops and leggings or tights. This will help keep you cozy and warm despite the chilly temperature.

Next, add a layer of protection from the elements with a heavy coat or parka. Look for one that's both stylish and functional, perhaps with a water-repellent finish to keep you dry in case of any unexpected rain showers.

Don't forget to accessorize with some warm and toasty gloves - you can never go wrong with a pair of luxurious leather gloves to add a touch of sophistication to your outfit. And, of course, don't neglect a stylish hat or scarf to add a pop of color and texture to your look.

Finally, consider adding some waterproof boots to complete your ensemble. Not only will they keep your feet dry, but they'll also add a chic touch to your ove

##Congratulation🎉 you have completed your course on how to use APIs in langGraph to make Agentic AI